In [2]:
# MUST RUN FIRST
import os, sys, platform, torch
os.environ["TRANSFORMERS_NO_TF"] = "1"              # avoid TF/Keras import issues
os.environ["HF_HUB_DISABLE_TF_WARNING"] = "1"

print("Python:", sys.version)
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))
    print("BF16 supported:", torch.cuda.is_bf16_supported())

ModuleNotFoundError: No module named 'torch'

In [2]:
!pip -q install "transformers>=4.43,<4.47" "accelerate>=0.30" datasets==2.21.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 47.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 134.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.6/177.6 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 126.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2024.6.1 which is incompatible.


In [3]:
import torch, os
torch.backends.cuda.matmul.allow_tf32 = True  # faster matmul on A100
torch.set_float32_matmul_precision("high")    # ok with BF16/TF32

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"  # fewer OOMs

In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

In [5]:
!pip install -q flash-attn --no-build-isolation

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 117.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [6]:
base = AutoModelForCausalLM.from_pretrained(
    "gpt2-medium",                          # or "gpt2" if you prefer small
    torch_dtype=torch.bfloat16,
    device_map="auto",
    attn_implementation="flash_attention_2" # A100-optimized
)
base.gradient_checkpointing_enable()
base.config.use_cache = False
print("GPT-2-medium loaded on A100 ✅")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/718 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.52G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

GPT-2-medium loaded on A100 ✅


In [7]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# Clean re-load with SDPA (fast and stable on A100)
tok = AutoTokenizer.from_pretrained("gpt2-medium")
tok.pad_token = tok.eos_token

base = AutoModelForCausalLM.from_pretrained(
    "gpt2-medium",
    torch_dtype=torch.bfloat16,
    device_map="auto",
    attn_implementation="sdpa",   # <- force SDPA; avoids FA2 runtime errors
)
base.gradient_checkpointing_enable()
base.config.use_cache = False
print("GPT-2-medium (SDPA) loaded ✅")


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

GPT-2-medium (SDPA) loaded ✅


In [8]:
from transformers import AutoTokenizer
tok = AutoTokenizer.from_pretrained("gpt2-medium"); tok.pad_token = tok.eos_token

prompt = "You are a helpful assistant. Q: What is RoPE in transformers? A:"
inputs = tok(prompt, return_tensors="pt").to("cuda")
out = base.generate(**inputs, max_new_tokens=64, do_sample=True, top_p=0.9)
print(tok.decode(out[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


You are a helpful assistant. Q: What is RoPE in transformers? A: We need it to perform this task, so we will rename it. Q: I think I've found a mistake in this FAQ. The RoPE acronym means, "Refined Amplification". A: It means the quality of amplification. A transformer is a capacitor that is designed to have a specific impedance for a given


In [9]:
prompt = "You are a helpful assistant. Q: What is RoPE in transformers? A:"
inputs = tok(prompt, return_tensors="pt").to("cuda")
with torch.inference_mode():
    out = base.generate(**inputs, max_new_tokens=64, do_sample=True, top_p=0.9)
print(tok.decode(out[0], skip_special_tokens=True))


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


You are a helpful assistant. Q: What is RoPE in transformers? A: RoPE transforms a transformer by applying electrical conductive compounds to the transformer's metal core. Q: What makes a coil transform? A: The coils are made up of two electrodes, and they are connected via a bridge coil to one another, called a resonant coil. Q: What do coil transformers do?


In [10]:
prompt = (
    "Question: In Transformers, what is Rotary Position Embedding (RoPE)? "
    "Answer in 2-3 sentences, plainly:\nA:"
)

inputs = tok(prompt, return_tensors="pt").to("cuda")
with torch.inference_mode():
    out = base.generate(
        **inputs,
        max_new_tokens=80,
        do_sample=False,           # <- greedy (deterministic)
        repetition_penalty=1.1,
        eos_token_id=tok.eos_token_id,
        pad_token_id=tok.eos_token_id,
    )
print(tok.decode(out[0], skip_special_tokens=True))


Question: In Transformers, what is Rotary Position Embedding (RoPE)? Answer in 2-3 sentences, plainly:
A: A rotational position of a vehicle's wheels. B) The rotation angle between the front and rear axles; C): The ratio that defines how much wheel travel each axle has to move before it starts rotating again. D). How far an object can be moved by one axis while still maintaining its original orientation on another axis without changing direction or speed. E), An aspect of movement where two axes


In [11]:
context = (
    "Context:\n"
    "Rotary Position Embedding (RoPE) encodes token positions by rotating query and key vectors "
    "in self-attention using sinusoidal phases at different frequency bands. This multiplicative "
    "scheme makes attention depend on relative offsets (m−n), allows flexible sequence lengths, "
    "and naturally induces distance-based decay.\n\n"
)

prompt = context + "Question: What is RoPE in Transformers?\nAnswer in 2-3 sentences:\nA:"
inputs = tok(prompt, return_tensors="pt").to("cuda")
with torch.inference_mode():
    out = base.generate(
        **inputs,
        max_new_tokens=90,
        do_sample=False,
        repetition_penalty=1.05,
        eos_token_id=tok.eos_token_id,
        pad_token_id=tok.eos_token_id,
    )
print(tok.decode(out[0], skip_special_tokens=True))


Context:
Rotary Position Embedding (RoPE) encodes token positions by rotating query and key vectors in self-attention using sinusoidal phases at different frequency bands. This multiplicative scheme makes attention depend on relative offsets (m−n), allows flexible sequence lengths, and naturally induces distance-based decay.

Question: What is RoPE in Transformers?
Answer in 2-3 sentences:
A: Rotating a vector with respect to the origin of the rotation can be done as follows: The first step is to rotate the position vector from 0 to 1 . Then we use an offset function that takes into account the direction of rotation for each iteration; this means that if you are looking up or down along the axis then it will take longer than when rotated left/right , but not necessarily faster. For example, say you have two rotations which


In [12]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_id = "tiiuae/falcon-7b-instruct"  # or "microsoft/phi-2" (general), or "HuggingFaceH4/zephyr-7b-beta"
tok = AutoTokenizer.from_pretrained(model_id)
tok.pad_token = tok.eos_token

base = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    attn_implementation="sdpa",
)

prompt = "What is Rotary Position Embedding (RoPE)? Answer briefly."
inputs = tok(prompt, return_tensors="pt").to("cuda")
with torch.inference_mode():
    out = base.generate(**inputs, max_new_tokens=120, do_sample=False, eos_token_id=tok.eos_token_id, pad_token_id=tok.eos_token_id)
print(tok.decode(out[0], skip_special_tokens=True))


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.48G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

What is Rotary Position Embedding (RoPE)? Answer briefly.
Rotary Position Embedding (RoPE) is a technique used in robotics to transform the position of an object in a 3D space to a specific coordinate system. It allows for easier communication and navigation between robots and can be used in various applications such as manufacturing and healthcare.
